In [1]:
# === SYSTEM & IMPORTS ===
# Block 5: Das Herzstück der Optimierung.
# Hier führen wir eine "Walk-Forward Cross-Validation" durch.
# Das bedeutet: Wir testen das Modell auf mehreren Zeiträumen, um sicherzustellen,
# dass es nicht nur zufällig in einem Monat gut war.
import os, sys, json, time, logging, glob, re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ROOT = os.path.abspath("..")
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

# TensorFlow soll nicht so viele Warnungen ausgeben
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

In [2]:
# === CONFIG & SETUP ===
with open(os.path.join(ROOT, "config.json"), "r") as f:
    C = json.load(f)

TICKER, START, END, INTERVAL = C["ticker"], C["start"], C["end"], C["interval"]
HORIZON  = int(C["horizon"])
LOOKBACK_DEFAULT = int(C["lookback"])
BATCH    = int(C.get("batch", 64))
SEED     = int(C.get("seed", 42))
FEATURESET = C.get("featureset", "v2")
EPS_MODE   = C.get("epsilon_mode", "abs")
EPSILON    = float(C.get("epsilon", 0.0005))
RESULTS_DIR = Path(C.get("results_dir", "../results"))

np.random.seed(SEED); tf.random.set_seed(SEED)

# Eigener Ordner für diesen Suchlauf
RUN_DIR = RESULTS_DIR / time.strftime("%Y-%m-%d_%H-%M-%S_wfcv")
(RUN_DIR / "plots").mkdir(parents=True, exist_ok=True)
print("WFCV_RUN_DIR:", RUN_DIR)

WFCV_RUN_DIR: ..\results\2026-01-01_18-18-53_wfcv


In [3]:
# === FAST MODE ===
# Option für schnelles Debugging (weniger Epochen, weniger Folds)
FAST = C.get("fast_wfcv", False)
EPOCHS_GRID = 60
N_FOLDS = 5
if FAST:
    EPOCHS_GRID = 25
    N_FOLDS = 3

In [4]:
# === DATEN LADEN ===
# Ähnliche Logik wie in den anderen Notebooks, um robuste Dateipfade zu finden
import yaml

yaml_path = f"../data/features_{FEATURESET}.yml"
meta = {}
label_h = label_mode = label_eps = None

if os.path.exists(yaml_path):
    with open(yaml_path, "r") as f:
        meta = yaml.safe_load(f) or {}
    lab = (meta or {}).get("label", {})
    label_h    = lab.get("horizon", None)
    label_mode = lab.get("mode", None)
    label_eps  = lab.get("epsilon", None)

def _parse_h_meps_from_name(path: str):
    mH = re.search(r"_cls_h(\d+)_", path)
    me = re.search(r"_(abs|rel)(\d+p\d+)\.csv$", path)
    H  = int(mH.group(1)) if mH else None
    md = me.group(1) if me else None
    eps= float(me.group(2).replace("p",".")) if me else None
    return H, md, eps

def _infer_from_existing_files(tkr, itv, start, end, mode_hint=None, eps_hint=None):
    pat = f"../data/{tkr}_{itv}_{start}_{end}_cls_h*_.csv".replace("_ .csv",".csv")
    cands = sorted(glob.glob(pat), key=os.path.getmtime)
    if mode_hint and (eps_hint is not None):
        tag = f"{mode_hint}{str(eps_hint).replace('.','p')}"
        cands = [c for c in cands if c.endswith(f"_{tag}.csv")]
    if not cands:
        return None
    return _parse_h_meps_from_name(cands[-1])

# Parameter bestimmen
H_FOR_FILE    = int(label_h)    if label_h    is not None else None
MODE_FOR_FILE = str(label_mode) if label_mode is not None else None
EPS_FOR_FILE  = float(label_eps) if label_eps is not None else None

if (H_FOR_FILE is None) or (MODE_FOR_FILE is None) or (EPS_FOR_FILE is None):
    inferred = _infer_from_existing_files(TICKER, INTERVAL, START, END,
                                          mode_hint=MODE_FOR_FILE, eps_hint=EPS_FOR_FILE)
    if inferred is not None:
        H_i, M_i, E_i = inferred
        H_FOR_FILE    = H_FOR_FILE    if H_FOR_FILE    is not None else H_i
        MODE_FOR_FILE = MODE_FOR_FILE if MODE_FOR_FILE is not None else M_i
        EPS_FOR_FILE  = EPS_FOR_FILE  if EPS_FOR_FILE  is not None else E_i

if (H_FOR_FILE is None) or (MODE_FOR_FILE is None) or (EPS_FOR_FILE is None):
    raise RuntimeError("Label-Definition unklar. Bitte Block 2 prüfen.")

# Dateipfad bauen
eps_tag   = f"{MODE_FOR_FILE}{str(EPS_FOR_FILE).replace('.','p')}"
TRAIN_CSV = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{H_FOR_FILE}_{eps_tag}.csv"

if not os.path.exists(TRAIN_CSV):
    # Fallback Suche nach Datei
    pat = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h*_{eps_tag}.csv"
    candidates = sorted(glob.glob(pat), key=os.path.getmtime)
    if candidates:
        TRAIN_CSV = candidates[-1]
    else:
        raise FileNotFoundError(f"CSV nicht gefunden: {TRAIN_CSV}")

print("Loaded TRAIN_CSV:", TRAIN_CSV)
df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=True).sort_index()

# Alle möglichen Features laden
OHLCV = {"open","high","low","close","volume"}
if meta:
    FEATURES_ALL = [c for c in meta.get("features", []) if c in df.columns]
else:
    FEATURES_ALL = [c for c in df.columns if c not in (OHLCV | {"target"})]
assert len(FEATURES_ALL) > 0, "Keine Features gefunden."

X_full = df[FEATURES_ALL].copy()
y_full = df["target"].astype(int).copy()

print("Label pos_rate:", round(y_full.mean(), 3), "| n:", len(y_full))

Loaded TRAIN_CSV: ../data/AAPL_1d_2010-01-01_2026-01-01_cls_h1_abs0p0005.csv
Label pos_rate: 0.514 | n: 3991


In [5]:
# === SPLITTING (Walk-Forward) ===
# Funktion teilt die Zeitreihe in wachsende Fenster (Folds).
# Fold 1: Trainier auf [0...T], Test auf [T...T+V]
# Fold 2: Trainier auf [0...T+V], Test auf [T+V...T+2V]
# ... usw.
def make_wf_splits(n, n_folds=5, val_frac=0.20, min_train_frac=0.45):
    val_len   = max(60, int(round(n * val_frac)))
    min_train = max(200, int(round(n * min_train_frac)))
    start_val_end = min_train + val_len
    if start_val_end + 1 > n:
        raise ValueError(f"Dataset zu kurz: {n}")
    
    # Endpunkte der Validation-Sets bestimmen
    val_ends = np.linspace(start_val_end, n, num=n_folds, endpoint=True).astype(int)
    val_ends = np.unique(val_ends)
    
    # Wenn zu wenige Punkte, dann fixe Schritte nehmen
    if len(val_ends) < n_folds:
        step = max(1, (n - start_val_end) // n_folds)
        val_ends = np.arange(start_val_end, start_val_end + step * n_folds, step)
        val_ends = np.clip(val_ends, start_val_end, n)
        
    stops = []
    for ve in val_ends[:n_folds]:
        te = int(ve - val_len)
        te = max(te, LOOKBACK_DEFAULT + 1)
        if te <= 0 or ve <= te or ve > n:
            continue
        # (Train-Slice, Valid-Slice)
        stops.append((slice(0, te), slice(te, ve)))
        
    if len(stops) != n_folds:
        raise RuntimeError(f"Erzeugte nur {len(stops)} von {n_folds} Folds.")
    return stops

n = len(df)
splits = make_wf_splits(n, n_folds=N_FOLDS, val_frac=0.20, min_train_frac=0.45)
print("Anzahl Folds:", len(splits))
if len(splits) > 0:
    tr_s, va_s = splits[0]
    print("  Fold0 sizes (train/val):", tr_s.stop, va_s.stop - va_s.start)

Anzahl Folds: 5
  Fold0 sizes (train/val): 1796 798


In [6]:
# === MODELL-HELPER ===
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef, average_precision_score, roc_auc_score
from tensorflow.keras import layers, regularizers, callbacks, optimizers, models

# 3D-Daten für RNN bauen
def make_windows(X_df, y_ser, lookback):
    Xv = X_df.values.astype(np.float32)
    yv = y_ser.values.astype(np.int32)
    xs, ys = [], []
    for i in range(lookback-1, len(X_df)):
        xs.append(Xv[i - lookback + 1 : i + 1])
        ys.append(yv[i])
    return np.stack(xs, axis=0), np.array(ys)

# Modellarchitektur generieren
def build_model(n_features, width1=64, width2=32, dropout=0.10, lr=5e-4, use_gru=True):
    rnn = layers.GRU if use_gru else layers.LSTM
    m = models.Sequential([
        layers.Input(shape=(None, n_features)),
        rnn(width1, return_sequences=True, recurrent_dropout=dropout),
        layers.LayerNormalization(),
        rnn(width2, recurrent_dropout=dropout),
        layers.LayerNormalization(),
        layers.Dense(16, activation="relu", kernel_regularizer=regularizers.l2(1e-5)),
        layers.Dense(1, activation="sigmoid"),
    ])
    m.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc"),
                 tf.keras.metrics.AUC(name="auprc", curve="PR")]
    )
    return m

# MCC maximieren: Sucht besten Threshold auf Validation
def mcc_at_best_thr(y_true, y_prob):
    ts = np.r_[0.0, np.unique(y_prob), 1.0]
    best = (-1.0, 0.5)
    for t in ts:
        m = matthews_corrcoef(y_true, (y_prob >= t).astype(int))
        if m > best[0]:
            best = (float(m), float(t))
    return best  # (mcc, thr)

# Training eines einzelnen Folds
def fit_eval_fold(X_tr, y_tr, X_va, y_va, lookback, hp, seed=SEED, batch=BATCH, epochs=EPOCHS_GRID):
    tf.keras.backend.clear_session()

    # Scaler nur auf Train fitten!
    scaler = StandardScaler()
    X_tr_s = pd.DataFrame(scaler.fit_transform(X_tr), index=X_tr.index, columns=X_tr.columns)
    X_va_s = pd.DataFrame(scaler.transform(X_va),     index=X_va.index, columns=X_va.columns)

    Xtr_win, ytr = make_windows(X_tr_s, y_tr, lookback)
    Xva_win, yva = make_windows(X_va_s, y_va, lookback)

    # Datasets erstellen
    ds_tr = tf.data.Dataset.from_tensor_slices((Xtr_win, ytr)) \
            .shuffle(len(Xtr_win), seed=seed) \
            .batch(batch).prefetch(tf.data.AUTOTUNE)
    ds_va = tf.data.Dataset.from_tensor_slices((Xva_win, yva)) \
            .batch(batch).prefetch(tf.data.AUTOTUNE)

    tf.keras.utils.set_random_seed(seed)
    model = build_model(
        n_features=Xtr_win.shape[-1],
        width1=hp["width1"], width2=hp["width2"],
        dropout=hp["dropout"], lr=hp["lr"], use_gru=(hp["cell"]=="GRU")
    )

    # Callbacks (Early Stopping, Learning Rate Reduction)
    cbs = [
        callbacks.EarlyStopping(monitor="val_auprc", mode="max", patience=6, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(monitor="val_auprc", mode="max", factor=0.5, patience=3, min_lr=1e-5),
        callbacks.TerminateOnNaN(),
    ]

    print(f"  -> fit (epochs={epochs}, batch={batch}) ...", flush=True)
    hist = model.fit(ds_tr, validation_data=ds_va, epochs=epochs, verbose=1 if FAST else 0, callbacks=cbs)

    # Evaluation
    yva_proba = model.predict(ds_va, verbose=0).ravel()
    mcc_val, thr_val = mcc_at_best_thr(yva, yva_proba)   # Beste Performance wählen
    yva_pred_best = (yva_proba >= thr_val).astype(int)

    metrics = dict(
        mcc=float(mcc_val),
        thr_val=float(thr_val),
        bal_acc=float(balanced_accuracy_score(yva, yva_pred_best)),
        auprc=float(average_precision_score(yva, yva_proba)),
        auroc=float(roc_auc_score(yva, yva_proba)),
        epochs_trained=int(len(hist.history["loss"]))
    )
    tf.keras.backend.clear_session()
    return metrics

In [7]:
# === SUCH-GRIDS DEFINIEREN ===
# Hier legen wir fest, welche Parameter-Kombinationen getestet werden sollen.
LOOKBACK_GRID = [40, 60, 120] if not FAST else [LOOKBACK_DEFAULT]
HP_GRID = [
    dict(width1=w1, width2=w2, dropout=dp, lr=lr, cell=cell)
    for (w1, w2) in [(32,16), (64,32)]
    for lr in [5e-4, 3e-4]
    for dp in [0.0, 0.1, 0.2]
    for cell in ["GRU", "LSTM"]
] if not FAST else [dict(width1=32, width2=16, dropout=0.10, lr=5e-4, cell="GRU")]

# Verschiedene Feature-Sets testen (z.B. nur Momentum? Oder auch Volatilität?)
FEATURE_SUBSETS = {
    "all": FEATURES_ALL,
    "mom_only": [c for c in FEATURES_ALL
                 if ("logret" in c) or ("macd" in c) or (c in {"sma_diff","rsi_14","bb_pos"})],
    "mom+vol": [c for c in FEATURES_ALL
                if (("logret" in c) or ("macd" in c) or (c in {"sma_diff","rsi_14","bb_pos"}))
                   or (c in {"realized_vol_10","vol_z_20"})]
}

print("Sanity check vor der Suche:")
print("  HP_GRID size:", len(HP_GRID))
print("  #splits:", len(splits))

Sanity check vor der Suche:
  HP_GRID size: 24
  #splits: 5


In [8]:
# === SUCHE DURCHFÜHREN ===
from time import perf_counter

print("Starte Suche ...", flush=True)

# Overrides für schnelle Tests innerhalb dieser Zelle
FAST_PATCH     = True
MAX_SECONDS    = 60 * 60  # Max 1 Stunde Laufzeit
EPOCHS_FAST    = 25
NFOLDS_FAST    = 3
BATCH_FAST     = 128

# Fallback Subsets für Fast Patch
FEATURE_SUBSETS_FAST = {}
if "mom+vol" in FEATURE_SUBSETS:
    FEATURE_SUBSETS_FAST["mom+vol"] = FEATURE_SUBSETS["mom+vol"]
elif "mom_only" in FEATURE_SUBSETS:
    FEATURE_SUBSETS_FAST["mom_only"] = FEATURE_SUBSETS["mom_only"]
else:
    k0 = next(iter(FEATURE_SUBSETS.keys()))
    FEATURE_SUBSETS_FAST[k0] = FEATURE_SUBSETS[k0]

LOOKBACK_GRID_FAST = [60]
HP_GRID_FAST = [dict(width1=32, width2=16, dropout=0.10, lr=5e-4, cell="GRU")]

# Patch anwenden
if FAST_PATCH:
    EPOCHS_GRID = EPOCHS_FAST
    splits = splits[:min(NFOLDS_FAST, len(splits))]
    FEATURE_SUBSETS = FEATURE_SUBSETS_FAST
    LOOKBACK_GRID = LOOKBACK_GRID_FAST
    HP_GRID = HP_GRID_FAST

csv_path = RUN_DIR / "wfcv_results.csv"
records = []
total_combos = 0
t0 = perf_counter()

# Resume-Logic: Wenn CSV existiert, schon berechnete überspringen
if csv_path.exists():
    done_df = pd.read_csv(csv_path)
    done_keys = {
        (r["features_used"], int(r["lookback"]),
         r["cell"], int(r["width1"]), int(r["width2"]),
         float(r["dropout"]), float(r["lr"]), int(r["fold"]))
        for _, r in done_df.iterrows()
    }
else:
    done_df = pd.DataFrame()
    done_keys = set()

stop_time = t0 + MAX_SECONDS

# HAUPTSCHLEIFE
for feat_name, FEATS in FEATURE_SUBSETS.items():
    if len(FEATS) == 0: continue

    for lookback in LOOKBACK_GRID:
        for hp in HP_GRID:
            total_combos += 1
            ran_folds = 0

            for fold_id, (tr_s, va_s) in enumerate(splits, start=1):
                key = (feat_name, int(lookback),
                       hp["cell"], int(hp["width1"]), int(hp["width2"]),
                       float(hp["dropout"]), float(hp["lr"]), int(fold_id))
                if key in done_keys:
                    ran_folds += 1
                    continue

                # Zeitbudget check
                if perf_counter() > stop_time:
                    print("[INFO] Zeitbudget erreicht — speichere & beende.")
                    out_df = pd.concat([done_df, pd.DataFrame.from_records(records)], ignore_index=True) if csv_path.exists() else pd.DataFrame.from_records(records)
                    out_df.to_csv(csv_path, index=False)
                    raise SystemExit(0)

                # Daten holen
                X_tr, y_tr = X_full.iloc[tr_s][FEATS], y_full.iloc[tr_s]
                X_va, y_va = X_full.iloc[va_s][FEATS], y_full.iloc[va_s]

                # Training
                mets = fit_eval_fold(X_tr, y_tr, X_va, y_va,
                                     lookback=lookback, hp=hp,
                                     epochs=EPOCHS_GRID, batch=BATCH_FAST)

                ran_folds += 1
                rec = {
                    "feature_set": FEATURESET,
                    "features_used": feat_name,
                    "n_features": len(FEATS),
                    "lookback": lookback,
                    **hp,
                    "fold": fold_id,
                    **mets
                }
                records.append(rec)

                # Inkrementell speichern
                cur_df = pd.DataFrame.from_records(records)
                out_df = pd.concat([done_df, cur_df], ignore_index=True)
                out_df.to_csv(csv_path, index=False)
                done_df = out_df

                print(f"[{feat_name} | LB={lookback} | Fold{fold_id}] MCC={mets['mcc']:.3f}")

t1 = perf_counter()
print(f"\nSuche fertig. Dauer={t1-t0:.1f}s")

# Final speichern
final_df = pd.read_csv(csv_path) if csv_path.exists() else pd.DataFrame.from_records(records)
final_df.to_csv(csv_path, index=False)

Starte Suche ...
  -> fit (epochs=25, batch=128) ...
[mom+vol | LB=60 | Fold1] MCC=0.120
  -> fit (epochs=25, batch=128) ...
[mom+vol | LB=60 | Fold2] MCC=0.096
  -> fit (epochs=25, batch=128) ...
[mom+vol | LB=60 | Fold3] MCC=0.119

Suche fertig. Dauer=32.0s


In [9]:
# === ERGEBNIS-ANALYSE ===
# Wir aggregieren die Ergebnisse aller Folds und suchen die beste Konfiguration
import pandas as pd, json, numpy as np

csv_path = RUN_DIR / "wfcv_results.csv"
results = pd.read_csv(csv_path)

# Duplikate entfernen
key_cols = ["features_used","lookback","cell","width1","width2","dropout","lr","fold"]
present_keys = [c for c in key_cols if c in results.columns]
results = results.drop_duplicates(subset=present_keys, keep="last").reset_index(drop=True)

# Gruppieren & Mittelwerte berechnen
agg_cols = [c for c in ["feature_set","features_used","n_features","lookback",
                        "width1","width2","dropout","lr","cell"] if c in results.columns]

agg_dict = {"mcc": ["mean","std"], "auprc": ["mean","std"], "auroc": ["mean"]}
g = results.groupby(agg_cols).agg(agg_dict)

# Flache Spaltennamen erzeugen
g.columns = [
    "_".join([str(x) for x in col if str(x) != ""]).strip("_")
    for col in g.columns.to_flat_index()
]
g = g.reset_index()

# Sortieren: Beste Konfiguration zuerst (nach MCC Mean)
g = g.sort_values(["mcc_mean","auprc_mean","mcc_std"], ascending=[False, False, True])

# Speichern
(g).to_csv(RUN_DIR / "wfcv_results_agg.csv", index=False)
top5 = g.head(5).copy()
top5.to_csv(RUN_DIR / "wfcv_results_top5.csv", index=False)

# Beste Config als JSON exportieren (für Step 3: Training)
best = top5.iloc[0].to_dict() if len(top5) else {}
with open(RUN_DIR / "best_config.json", "w") as f:
    json.dump(best, f, indent=2)

print("Best config:", best)

Best config: {'feature_set': 'v2', 'features_used': 'mom+vol', 'n_features': 11, 'lookback': 60, 'width1': 32, 'width2': 16, 'dropout': 0.1, 'lr': 0.0005, 'cell': 'GRU', 'mcc_mean': 0.11192501701210433, 'mcc_std': 0.013737426269908473, 'auprc_mean': 0.5558566616457216, 'auprc_std': 0.02613838967568943, 'auroc_mean': 0.5252876312581236}


In [10]:
# === VISUALISIERUNG: HEATMAPS ===
import pandas as pd
import matplotlib.pyplot as plt

agg_path = RUN_DIR / "wfcv_results_agg.csv"
agg = pd.read_csv(agg_path)

pivot_index = "lookback" if "lookback" in agg.columns else None
col_candidates = ["features_used", "cell", "width1"]
pivot_columns = [c for c in col_candidates if c in agg.columns]
if not pivot_columns: pivot_columns = [agg.columns[0]]

(RUN_DIR / "plots").mkdir(parents=True, exist_ok=True)

def _plot_grid(df: pd.DataFrame, value_col: str, fname: str):
    if value_col not in df.columns: return
    idx = pivot_index or pivot_columns[0]
    pvt = df.pivot_table(index=idx, columns=pivot_columns, values=value_col, aggfunc="mean")

    plt.figure(figsize=(10, 5))
    im = plt.imshow(pvt.values, aspect="auto")
    plt.colorbar(im)
    plt.title(fname.replace("_", " ").replace(".png", ""))
    plt.tight_layout()
    plt.savefig(RUN_DIR / "plots" / fname, dpi=160)
    plt.close()

_plot_grid(agg, "mcc_mean",   "score_grid_mcc.png")
_plot_grid(agg, "auprc_mean", "score_grid_auprc.png")
print("Grids gespeichert.")

Grids gespeichert.


In [11]:
# === VISUALISIERUNG: BOXPLOTS ===
# Zeigt die Streuung der Leistung über die verschiedenen Folds
def _short_label(r):
    return f"{r['features_used']}-{r['cell']}-{int(r['width1'])}/{int(r['width2'])}-lb{int(r['lookback'])}-dp{r['dropout']}-lr{r['lr']}"

if not results.empty:
    results["config_label"] = results.apply(_short_label, axis=1)

    # MCC Plot
    plt.figure(figsize=(max(8, 0.35*len(results["config_label"].unique())), 5))
    data = [grp["mcc"].values for _, grp in results.groupby("config_label")]
    labels = list(results.groupby("config_label").groups.keys())
    plt.boxplot(data, showmeans=True, meanline=True)
    plt.xticks(range(1, len(labels)+1), labels, rotation=90)
    plt.title("MCC über Folds (pro Konfiguration)")
    plt.tight_layout()
    plt.savefig(RUN_DIR / "plots" / "boxplots_mcc.png", dpi=160)
    plt.close()

In [12]:
# === INFO-DUMP ===
# Speichert Metadaten über den Lauf
run_info = {
    "seed": SEED,
    "epochs_grid": EPOCHS_GRID,
    "n_folds": N_FOLDS,
    "val_frac": 0.20,
    "min_train_frac": 0.45,
    "lookback_grid": LOOKBACK_GRID,
    "hp_grid_size": (len(HP_GRID) if not FAST else 1),
    "feature_subsets": list(FEATURE_SUBSETS.keys()),
    "train_csv": TRAIN_CSV,
    "label_resolution": {
        "source": "yaml" if os.path.exists(yaml_path) and (label_h is not None) else "inferred_from_csv",
        "yaml_path": yaml_path
    },
    "labels": {"horizon": H_FOR_FILE, "mode": MODE_FOR_FILE, "epsilon": EPS_FOR_FILE}
}
with open(RUN_DIR / "wfcv_run_info.json", "w") as f:
    json.dump(run_info, f, indent=2)

print("\nBlock 5 abgeschlossen. Ergebnisse in:", RUN_DIR)


Block 5 abgeschlossen. Ergebnisse in: ..\results\2026-01-01_18-18-53_wfcv
